In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime

from win32com.client import Dispatch
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import ssl

def speak(str1):
    speak=Dispatch(("SAPI.SpVoice"))
    speak.Speak(str1)

def send_email(to_email, subject, body, attachment_path=None):
    sender_email = "youremail@gmail.com"
    password = "your_app_password"  # Replace with your App Password

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    if attachment_path:
        with open(attachment_path, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={os.path.basename(attachment_path)}")
        msg.attach(part)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.send_message(msg)

video=cv2.VideoCapture(0)
facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

with open('data/names.pkl', 'rb') as w:
    LABELS=pickle.load(w)
with open('data/faces_data.pkl', 'rb') as f:
    FACES=pickle.load(f)

print('Shape of Faces matrix --> ', FACES.shape)

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

imgBackground=cv2.imread("background.png")

COL_NAMES = ['NAME', 'TIME']

while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img=cv2.resize(crop_img, (50,50)).flatten().reshape(1,-1)
        output=knn.predict(resized_img)
        ts=time.time()
        date=datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp=datetime.fromtimestamp(ts).strftime("%H:%M-%S")
        exist=os.path.isfile("Attendance/Attendance_" + date + ".csv")
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
        cv2.putText(frame, str(output[0]), (x,y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
        attendance=[str(output[0]), str(timestamp)]
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame",imgBackground)
    k=cv2.waitKey(1)
    if k==ord('o'):
        speak("Attendance Taken..")
        time.sleep(3)
        if exist:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(attendance)
            csvfile.close()
        else:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
            csvfile.close()

        # Email sending code starts here
        with open('data/Emails.pkl', 'rb') as f:
            EMAILS = pickle.load(f)
        name = output[0]
        if name in LABELS:
            email_index = LABELS.index(name)
            recipient_email = EMAILS[email_index]
            send_email(
                to_email=recipient_email,
                subject="Attendance Confirmation",
                body=f"Hello {name}, your attendance has been successfully recorded on {date} at {timestamp}.",
                attachment_path=f"Attendance/Attendance_{date}.csv"
            )
        # Email sending code ends here

    if k==ord('q'):
        break
video.release() 
cv2.destroyAllWindows()


In [ ]:

from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl

def speak(str1):
    speak=Dispatch(("SAPI.SpVoice"))
    speak.Speak(str1)

def send_email(to_email, subject, body):
    sender_email = "infopiyush2004@gmail.com"
    password = "hduoatuqjbijoiim" 

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    context = ssl.create_default_context()
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.ehlo()
            server.starttls(context=context)
            server.ehlo()
            server.login(sender_email, password)
            server.send_message(msg)
    except Exception as e:
        print(f"Failed to send email: {e}")


video=cv2.VideoCapture(0)
facedetect=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

with open('data/names.pkl', 'rb') as w:
    LABELS=pickle.load(w)
with open('data/faces_data.pkl', 'rb') as f:
    FACES=pickle.load(f)

print('Shape of Faces matrix --> ', FACES.shape)

FACES = FACES / 255.0 
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

imgBackground=cv2.imread("background.png")
COL_NAMES = ['NAME', 'TIME']
while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1) / 255.0
        
        distances, indices = knn.kneighbors(resized_img)
      # print("Distances:", distances, "Output:", output)
        threshold = 9                                                                      # Adjust based on your data
        if distances[0][0] > threshold:
            output = ["Unknown"]
        else:
            output = knn.predict(resized_img)

        
        ts=time.time()
        date=datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp=datetime.fromtimestamp(ts).strftime("%H:%M:%S")
        exist=os.path.isfile("Attendance/Attendance_" + date + ".csv")
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(50,50,255),2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
        cv2.putText(frame, str(output[0]), (x,y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 1)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (50,50,255), 1)
        attendance=[str(output[0]), str(timestamp)]
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame",imgBackground)
    k=cv2.waitKey(1)
    if k == ord('o') and output[0] != "Unknown":
        speak("Attendance Taken..")
        time.sleep(3)
        if exist:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(attendance)
            csvfile.close()
        else:
            with open("Attendance/Attendance_" + date + ".csv", "+a") as csvfile:
                writer=csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
            csvfile.close()

        with open('data/Emails.pkl', 'rb') as f:
            EMAILS = pickle.load(f)
        name = output[0]
        if name in LABELS:
            email_index = LABELS.index(name)
            recipient_email = EMAILS[email_index]
            send_email(
                to_email=recipient_email,
                subject="Attendance Confirmation",
                body=f"Hello {name}, your attendance has been successfully recorded on {date} at {timestamp}.")

    if k==ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [ ]:

#GUI Attandeane View



import tkinter as tk
from tkinter import messagebox
import subprocess
import os
import sys

def run_email_sample():
    name = entry_name.get()
    Email = entry_email.get()
    
    if not name or not Email:
        messagebox.showwarning("Input Error", "Please enter both Name and Email")
        return
    
    messagebox.showinfo("Running", f"Starting Face collection for {name}) ...")
    
    try:       
        env = os.environ.copy()
        env.update({"NAME": name, "EMAIL": Email})
        
        subprocess.run([
            sys.executable, "-m", "jupyter", "nbconvert",
            "--execute",
            "--to", "notebook",
            "--inplace", "Face_Email_View_sample.ipynb"
        ],
        check=True,
        env=env)
        
        messagebox.showinfo("Success", "Face added successfully!")
        
    except Exception as e:
        messagebox.showerror("Error", f"Execution failed:\n{e}")

def run_email_recognition():
    messagebox.showinfo("Running", "Starting Attendance process...")
    
    try:
        subprocess.run([
            sys.executable, "-m", "jupyter", "nbconvert",
            "--execute",
            "--to", "notebook",
            "--inplace", "Face_Email_View_recognition.ipynb"
        ], check=True)
        
        messagebox.showinfo("Success", "Attendance Taken successfully!")
        
    except Exception as e:
        messagebox.showerror("Error", f"Execution failed:\n{e}")

def View_attendance():
    messagebox.showinfo("Running", "viewing Attendance ...")
    
    try:
        subprocess.run([
            sys.executable, "-m", "jupyter", "nbconvert",
            "--execute",
            "--to", "notebook",
            "--inplace", "view_attandence.ipynb"
        ], check=True)
        
        messagebox.showinfo("Success", "Attendance viewed successfully!")
        
    except Exception as e:
        messagebox.showerror("Error", f"Execution failed:\n{e}")



def reset_entry():
    entry_name.config(text="Enter Your Name:")   
    entry_email.config(text="Enter Your Email:") 
    entry_name.delete(0, tk.END)                
    entry_email.delete(0, tk.END)

def exit_app():
    confirm = messagebox.askyesno("Exit", "Are you sure you want to exit?")
    if confirm:
        root.destroy()
        

# GUI Setup
root = tk.Tk()
root.title("Face Recognition Dashboard")
root.geometry("400x450")
root.configure(bg="#f0f0f0")

tk.Label(root, text="Enter Your Name:",fg="#076834",font=("Helvetica", 15, "bold"), bg="#f0f0f0").pack(pady=5)
entry_name = tk.Entry(root,bg="#fdffb6", width=35)
entry_name.pack(pady=5)

tk.Label(root, text="Enter Your Email:",fg="#076834",font=("Helvetica", 15, "bold"), bg="#f0f0f0").pack(pady=5)
entry_email = tk.Entry(root,bg="#fdffb6", width=35)
entry_email.pack(pady=5)

tk.Button(root, text="Reset",bg="#FFD700", width=8, height=1,font=("Helvetica", 9, "bold"), command=reset_entry).pack(pady=10)

tk.Button(root, text="Add Faces", bg="#4CAF50", fg="white", width=25, height=2,font=("Helvetica", 10, "bold"),
          command=run_email_sample).pack(pady=10)
tk.Button(root, text="Take Attendance", bg="#2196F3", fg="white", width=22, height=2,font=("Helvetica", 10, "bold"),
          command=run_email_recognition).pack(pady=10)

tk.Button(root, text="View Attendance", bg="#736b73", fg="white", width=17, height=2,font=("Helvetica", 10, "bold"),
          command=View_attendance).pack(pady=10)
# Exit button
tk.Button(root, text="Exit", bg="#f44336", fg="white", width=10, height=1,font=("Helvetica", 10, "bold"),
          command=exit_app).pack(pady=10)

root.mainloop()

In [ ]:
# Check the shapes before appending
print(faces.shape)  # Existing data
print(faces_data.shape)  # New data

if faces.shape[1] != faces_data.shape[1]:
    # Shapes mismatch; cannot append!
    # You need to reshape all data to the same feature vector size,
    # likely standardize old data or drop it if incompatible.
    print("Feature vector shape mismatch; cannot append!")
else:
    faces = np.append(faces, faces_data, axis=0)
